In [37]:
# This file contains various functions needed that doesn't fit into the other files
import numpy as np


def Interpolate_gaps_in_plankton_positions(plankton_positions):
    no_of_timesteps, no_of_cols = np.shape(plankton_positions)
    for i in range(no_of_cols):
        for j in range(1,no_of_timesteps-1):
            if np.isnan(plankton_positions[j,i]) and np.any(np.isnan([plankton_positions[j-1,i], plankton_positions[j+1,i]]))==False:
                plankton_positions[j,i] = (plankton_positions[j-1,i] + plankton_positions[j+1,i])/2
    return plankton_positions


In [41]:
from scipy.ndimage import label
def get_blob_center(label,array):
    x, y = np.where(array==label)
    x_center = np.sum(x)/len(x)
    y_center = np.sum(y)/len(y)
    
    return x_center, y_center


def get_blob_centers(prediction):
    prediction[prediction<0.7]=0
    labeled_array, num_features = label(prediction, structure = [[1,1,1],
                                                                 [1,1,1],
                                                                 [1,1,1]])
    centers = get_blob_center(1, labeled_array)
    for i in range(2,num_features):
        if np.count_nonzero(labeled_array==(i)) > 0:
            centers = np.vstack((centers,get_blob_center(i, labeled_array)))
    return centers

def Extract_positions_from_prediction(im_stack, model, layer):
    prediction = model.predict(im_stack)[0, :, :, layer]
    positions = get_blob_centers(prediction)
    return positions

In [260]:
def RunningAverage(folder_path, tot_no_of_frames, center_frame):
    import os  
    from deeptrack.features import LoadImage, Feature
    import numpy as np
    list_paths = os.listdir(folder_path)
    first_file_format = list_paths[0][-3:]
    for i in range(len(list_paths)):
        if list_paths[i][-3:] != first_file_format:
            print('Only the images to be analyzed can be in the folder with the images.')
            break
    frames_one_dir = int(tot_no_of_frames/2)
    first_image = np.asarray(LoadImage(folder_path +'\\' + list_paths[0]).resolve())
    mean_image = np.zeros(first_image.shape)
    start_point, end_point = max(center_frame - frames_one_dir,0), min(center_frame + frames_one_dir + 1, len(list_paths))
    for i in range(start_point, end_point):
        mean_image += np.asarray(LoadImage(folder_path +'\\' + list_paths[max(i,0)]).resolve()) / tot_no_of_frames
    
    return mean_image

In [264]:
def Normalize_image(image, min_value=0, max_value=1):
    min_im = np.min(image)
    max_im = np.max(image)
    image = image/(max_im-min_im) * (max_value - min_value)
    return image - np.min(image) + min_value

In [6]:
from scipy.spatial.distance import cdist
class Plankton:
    def __init__(self, position, number_of_timesteps, current_timestep):
        self.positions = np.zeros(shape = (number_of_timesteps, 2), dtype='object')*np.nan
        self.positions[current_timestep,:] = position
        self.number_of_timesteps = number_of_timesteps
    
    def add_position(self, position, timestep):
        self.positions[timestep,:] = position
         
    def get_latest_position(self, timestep, threshold):
        latest_position = [np.nan, np.nan]
        for i in range(self.number_of_timesteps - timestep, min(threshold + self.number_of_timesteps - timestep, self.number_of_timesteps+1)):
            if np.isfinite(self.positions[-i,0]):
                latest_position = self.positions[-i,:]
                break
        return latest_position
    
            
    def get_mean_velocity(self):
        no_of_timesteps = len(self.positions[:,0])
        no_of_numbers = sum(x is not np.nan for x in self.positions[:,0].tolist())
        mean_velocity = 0
        if no_of_numbers > 1:
            for i in range(no_of_timesteps-1):
                mean_velocity += np.nansum(np.linalg.norm(self.positions[i,:]-self.positions[i+1,:]))/(no_of_numbers)
        else:
            mean_velocity = 0
        return mean_velocity
        
def Initialize_plankton(positions, number_of_timesteps):
    no_of_plankton = np.shape(positions)[0]
    list_of_plankton = []
    
    for i in range(no_of_plankton):
        globals()['plankton%d' % i] = Plankton(positions[i,:], number_of_timesteps, 0)
        list_of_plankton.append(globals()['plankton%d' % i])
    return list_of_plankton

def Update_list_of_plankton(list_of_plankton, positions, max_dist, timestep, threshold, extrapolate):
    if np.any(np.isnan(positions)):
        return list_of_plankton
    
    no_of_plankton = len(list_of_plankton)
    no_of_positions = len(positions)
    plankton_positions = np.zeros([no_of_plankton, 2])
    
    for i in range(no_of_plankton):
        plankton_positions[i,:] = list_of_plankton[i].get_latest_position(timestep = timestep, threshold = threshold)

    
    if extrapolate == True and timestep > 1:
        plankton_positions = Extrapolate_positions(plankton_positions)
        
    if len(positions.shape)==1:
        positions = np.reshape(positions, (-1, 2))
        no_of_positions = len(positions)
    
    distances = cdist(positions, plankton_positions)
    
    for i in range(no_of_positions):
        if np.nanmin(distances[i,:]) > max_dist:
            position = positions[i,:]
            globals()['plankton%d' % (len(list_of_plankton))] = Plankton(position, plankton0.number_of_timesteps, timestep)
            list_of_plankton.append(globals()['plankton%d' % (len(list_of_plankton))])
        else:
            if np.sum(distances[i,:] < max_dist) > 1:
                temp_indices = np.where(distances[i,:] < max_dist)[0]
                temp_dists = distances[i,:][temp_indices]
                temp_veldiffs = np.zeros((len(temp_indices),1))
                for j in range(len(temp_dists)):
                    temp_veldiffs[j] = np.abs(temp_dists[j]-list_of_plankton[temp_indices[j]].get_mean_velocity())
                temp_min_vel = np.min(temp_veldiffs)
                temp_min_index = np.where(temp_veldiffs==temp_min_vel)[0][0]
                list_of_plankton[temp_indices[temp_min_index]].add_position(positions[i,:], timestep)
                
            else:
                temp_min_dist = np.nanmin(distances[i,:])
                temp_min_index = np.nonzero(distances[i,:]==temp_min_dist)[0][0]
                list_of_plankton[temp_min_index].add_position(positions[i,:], timestep)
    return list_of_plankton

def Interpolate_gaps_in_plankton_positions(plankton_positions):
    no_of_timesteps, no_of_cols = np.shape(plankton_positions)
    for i in range(no_of_cols):
        for j in range(1,no_of_timesteps-1):
            if np.isnan(plankton_positions[j,i]) and np.any(np.isnan([plankton_positions[j-1,i], plankton_positions[j+1,i]]))==False:
                plankton_positions[j,i] = (plankton_positions[j-1,i] + plankton_positions[j+1,i])/2
    return plankton_positions

def Extrapolate_positions(plankton_positions):
    no_of_timesteps, no_of_cols = np.shape(plankton_positions)
    for i in range(no_of_cols):
        for j in range(1,no_of_timesteps):
            if np.isnan(plankton_positions[j,i]) and np.any(np.isnan([plankton_positions[j-1,i], plankton_positions[j-2,i]]))==False:
                plankton_positions[j,i] = (2 * plankton_positions[j-1,i] - plankton_positions[j-2,i])
    return plankton_positions



def Trim_positions_from_stationary_planktons(positions, min_distance):
    no_of_timesteps, no_of_plankton = positions[:,0::2].shape
    cols_to_delete = []
    
    for j in range(no_of_plankton):
        temp_dist=0
        temp_positions = positions[:,2*j:2*j+1]
        
        for i in range(len(temp_positions[:,0])-1):
            temp_dist += np.nansum(np.linalg.norm(temp_positions[i,:]-temp_positions[i+1,:]))
        temp_dist = np.nansum(temp_dist)
        
        if temp_dist < min_distance:
            cols_to_delete.append(2*j)
            cols_to_delete.append(2*j+1)
        
    new_positions = np.delete(positions,cols_to_delete,1)
    return new_positions



In [ ]:
import cv2
import numpy as np
import glob

def Make_video(path_frames, save_path, fps, no_of_frames):
    # Make sure that the path_frames name looks like ...\\frame%d.jpg (if the frames
    # are named frame0, frame1, frame2...) or ...\\frame%3d.jpg (if the frames are named frame001, frame002...) 
    # so that the counter works.
    img_array = []
    for i in range(no_of_frames):
    
        img = cv2.imread(path_frames % i)

        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
        print(i)

    out = cv2.VideoWriter(save_path ,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()